In [5]:
import sys
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import os
# import wandb
from transformers import ViTMAEConfig

from custom_models.CustomViT import CustomViT
from custom_models.CustomViTMAE import CustomViTMAE
from transformers.models.vit_mae.modeling_vit_mae import ViTMAEModel
# from tem_dataloader import MultimodalDatasetPerTrajectory
from torch.utils.data import DataLoader

from d3rlpy.algos import CQL
from d3rlpy.dataset import Episode, MDPDataset, Transition
# wandb.login() 

ModuleNotFoundError: No module named 'wandb'

In [7]:
from custom_models.CustomViT import CustomViT
from custom_models.CustomViTMAE import CustomViTMAE
import torch
# call CustomViT
from transformers import AutoImageProcessor, ViTMAEForPreTraining, ViTMAEConfig
from PIL import Image

output_dir='/home/ubuntu/camelmera'
# trained_model_name = 'multimodal'
# output_dir='/home/ubuntu/weights/' + trained_model_name

# Initialize a new CustomViT model
model_name = "facebook/vit-mae-base"
vit_config = ViTMAEConfig.from_pretrained(model_name)
vit_config.output_hidden_states=True
vit_model = CustomViT(config=vit_config)

# Initialize a new CustomViTMAE model
model_name = "facebook/vit-mae-base"
config = ViTMAEConfig.from_pretrained(model_name)
config.output_hidden_states=True
custom_model = CustomViTMAE(config=config)
custom_model.vit = vit_model

# Load the state_dict from the saved model
state_dict = torch.load(f"{output_dir}/pytorch_model.bin")
custom_model.load_state_dict(state_dict)

# don't need decoders
vit_encoder = custom_model.vit

In [2]:
from tem_dataloader import MultimodalDataset
import functools

environment_name = 'AbandonedFactoryExposure'
environemnt_directory = f'/mnt/temp_mount/{environment_name}/Data_easy'
dataset = MultimodalDataset(environemnt_directory)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P001
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P004
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P006
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P005
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P008
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P002
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P009
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P003
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P000
Number of images: 8185
Number 

In [13]:
import numpy as np

def reward_function(state_embedding, goal_embedding, threshold=0.05, goal_reward=100):
    distance = np.linalg.norm(state_embedding - goal_embedding)

    if distance <= threshold:
        # Give a large positive reward when the goal is reached
        reward = goal_reward
    else:
        # Give a negative reward proportional to the distance otherwise
        reward = -distance

    return reward

In [14]:
from tem_dataloader import MultimodalDatasetPerTrajectory
import functools
import os
from torch.utils.data import Dataset, DataLoader

environment_name = 'AbandonedFactoryExposure'
environemnt_directory = f'/mnt/temp_mount/{environment_name}/Data_easy'
OBSERVATION_SIZE = 768
ACTION_SIZE = 7
BATCH_SIZE = 64

for i in range(0,10):
    if i==7:
        continue
    trajectory_folder_path = os.path.join(environemnt_directory, f'P00{i}')
    my_dataset = MultimodalDatasetPerTrajectory(trajectory_folder_path)
    train_dataloader = DataLoader(my_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Initialize empty arrays for observations, actions, rewards, and terminals
    all_observations = np.empty((0, OBSERVATION_SIZE))
    all_actions = np.empty((0, ACTION_SIZE))
    all_rewards = np.empty(0)
    all_terminals = np.empty(0, dtype=bool)

    for batch_idx, data in enumerate(train_dataloader):
        # get embedding
        vit_encoder.cuda()
        vit_encoder.eval()
        pixel_values = data["pixel_values"].cuda()
        pixel_values1 = data["pixel_values1"].cuda()
        pixel_values2 = data["pixel_values2"].cuda()
        outputs = vit_encoder(pixel_values,pixel_values1,pixel_values2,noise=None)
        embedding = outputs.last_hidden_state[:,0,:]
        observation = embedding.cpu().detach().numpy()
        # get action
        pose = data["pose_values"]
        action = torch.diff(pose,axis = 0).numpy()
        action = np.concatenate((action, np.zeros((1,7))), axis=0)
        # get reward
        goal = observation[-1]
        partial_function = functools.partial(reward_function, goal_embedding=goal)
        reward = np.apply_along_axis(partial_function, 1, observation)
        # get terminals
        terminals = np.zeros_like(reward, dtype=int)
        terminals[reward == 100] = 1

        # Concatenate observations, actions, rewards, and terminals
        all_observations = np.vstack((all_observations, observation))
        all_actions = np.vstack((all_actions, action))
        all_rewards = np.hstack((all_rewards, reward))
        all_terminals = np.hstack((all_terminals, terminals))

    print("All observations shape:", all_observations.shape)
    print("All actions shape:", all_actions.shape)
    print("All rewards shape:", all_rewards.shape)
    print("All terminals shape:", all_terminals.shape)
    
    np.save(f'trajectories/all_observations_P00{i}.npy', all_observations)
    np.save(f'trajectories/all_actions_P00{i}.npy', all_actions)
    np.save(f'trajectories/all_rewards_P00{i}.npy', all_rewards)
    np.save(f'trajectories/all_terminals_P00{i}.npy', all_terminals)

Processing folder: /mnt/temp_mount/AbandonedFactoryExposure/Data_easy/P000
Number of images: 1480
Number of depth: 1480
Number of lidar: 1480
Number of pose: 1480
All observations shape: (1480, 768)
All actions shape: (1480, 7)
All rewards shape: (1480,)
All terminals shape: (1480,)
Processing folder: /mnt/temp_mount/AbandonedFactoryExposure/Data_easy/P001
Number of images: 1055
Number of depth: 1055
Number of lidar: 1055
Number of pose: 1055
All observations shape: (1055, 768)
All actions shape: (1055, 7)
All rewards shape: (1055,)
All terminals shape: (1055,)
Processing folder: /mnt/temp_mount/AbandonedFactoryExposure/Data_easy/P002
Number of images: 698
Number of depth: 698
Number of lidar: 698
Number of pose: 698
All observations shape: (698, 768)
All actions shape: (698, 7)
All rewards shape: (698,)
All terminals shape: (698,)
Processing folder: /mnt/temp_mount/AbandonedFactoryExposure/Data_easy/P003
Number of images: 1009
Number of depth: 1009
Number of lidar: 1009
Number of pose

In [22]:
from tem_dataloader import MultimodalDatasetPerTrajectory
# environment_name = 'AmericanDinerExposure'
# environemnt_directory = f'/media/tyz/3B6FFE7354FF3296/11_777/tartanairv2filtered/{environment_name}/Data_easy'
# my_dataset = MultimodalDatasetPerTrajectory(environemnt_directory)
environment_name = 'AbandonedFactoryExposure'
environemnt_directory = f'/mnt/data/tartanairv2filtered/{environment_name}/Data_easy'
OBSERVATION_SIZE = 768
ACTION_SIZE = 7
BATCH_SIZE = 64

# for folder in os.listdir(environemnt_directory):
trajectory_folder_path = os.path.join(environemnt_directory, 'P000')
my_dataset = MultimodalDatasetPerTrajectory(trajectory_folder_path)
train_dataloader = DataLoader(my_dataset, batch_size=BATCH_SIZE, shuffle=False)

Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P000
Number of images: 1480
Number of depth: 1480
Number of lidar: 1480
Number of pose: 1480


In [24]:
import functools
import numpy as np
import torch

# Initialize empty arrays for observations, actions, rewards, and terminals
all_observations = np.empty((0, OBSERVATION_SIZE))
all_actions = np.empty((0, ACTION_SIZE))
all_rewards = np.empty(0)
all_terminals = np.empty(0, dtype=bool)

for batch_idx, data in enumerate(train_dataloader):
    # get embedding
    vit_encoder.cuda()
    vit_encoder.eval()
    pixel_values = data["pixel_values"].cuda()
    pixel_values1 = data["pixel_values1"].cuda()
    pixel_values2 = data["pixel_values2"].cuda()
    outputs = vit_encoder(pixel_values,pixel_values1,pixel_values2,noise=None)
    embedding = outputs.last_hidden_state[:,0,:]
    observation = embedding.cpu().detach().numpy()
    # get action
    pose = data["pose_values"]
    action = torch.diff(pose,axis = 0).numpy()
    action = np.concatenate((action, np.zeros((1,7))), axis=0)
    # get reward
    goal = observation[-1]
    partial_function = functools.partial(reward_function, goal_embedding=goal)
    reward = np.apply_along_axis(partial_function, 1, observation)
    # get terminals
    terminals = [False]*BATCH_SIZE
    terminals[-1]=True
    terminals = np.array(terminals)
    if batch_idx==0:
        print(observation.shape)
        print(action.shape)
        print(reward.shape)
        print(terminals.shape)

(64, 768)
(64, 7)
(64,)
(64,)


In [25]:
print("All observations shape:", all_observations.shape)
print("All actions shape:", all_actions.shape)
print("All rewards shape:", all_rewards.shape)
print("All terminals shape:", all_terminals.shape)

All observations shape: (1480, 768)
All actions shape: (1480, 7)
All rewards shape: (1480,)
All terminals shape: (1536,)


In [17]:
np.save('all_observations.npy', all_observations)
np.save('all_actions.npy', all_actions)
np.save('all_rewards.npy', all_rewards)
np.save('all_terminals.npy', all_terminals)

In [15]:
'''
Args:
        observations (numpy.ndarray): N-D array. If the
            observation is a vector, the shape should be
            `(N, dim_observation)`. If the observations is an image, the shape
            should be `(N, C, H, W)`.
        actions (numpy.ndarray): N-D array. If the actions-space is
            continuous, the shape should be `(N, dim_action)`. If the
            action-space is discrete, the shape should be `(N,)`.
        rewards (numpy.ndarray): array of scalar rewards. The reward function
            should be defined as :math:`r_t = r(s_t, a_t)`.
        terminals (numpy.ndarray): array of binary terminal flags.
        episode_terminals (numpy.ndarray): array of binary episode terminal
            flags. The given data will be splitted based on this flag.
            This is useful if you want to specify the non-environment
            terminations (e.g. timeout). If ``None``, the episode terminations
            match the environment terminations.
        discrete_action (bool): flag to use the given actions as discrete
            action-space actions. If ``None``, the action type is automatically
            determined.
    '''
cql_dataset = MDPDataset(observations=all_observations,actions=all_actions,rewards=all_rewards,terminals=all_terminals,episode_terminals=all_terminals)

In [16]:
from d3rlpy.algos import CQL

# setup CQL algorithm
cql = CQL(use_gpu=True)

# split train and test episodes
# train_episodes, test_episodes = train_test_split(cql_dataset, test_size=0.25)

# start training
cql.fit(cql_dataset,
        eval_episodes=None,
        n_epochs=10,
        scorers=None)

2023-05-04 01:27:46 [debug    ] RoundIterator is selected.
2023-05-04 01:27:46 [info     ] Directory is created at d3rlpy_logs/CQL_20230504012746
2023-05-04 01:27:46 [debug    ] Building models...
2023-05-04 01:27:46 [debug    ] Models have been built.
2023-05-04 01:27:46 [info     ] Parameters are saved to d3rlpy_logs/CQL_20230504012746/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 0.0001, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_threshold': 10.0, 'batch_size': 256, 'conservative_weight': 5.0, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rat

Epoch 1/10:   0%|          | 0/31 [00:00<?, ?it/s]

2023-05-04 01:27:48 [info     ] CQL_20230504012746: epoch=1 step=31 epoch=1 metrics={'time_sample_batch': 0.0009980278630410471, 'time_algorithm_update': 0.08748865127563477, 'temp_loss': 11.744194215343844, 'temp': 0.9984005458893315, 'alpha_loss': -31.277802190473004, 'alpha': 1.0016018421419206, 'critic_loss': 3721.7289094002017, 'actor_loss': -10.316245063658684, 'time_step': 0.0886292073034471} step=31
2023-05-04 01:27:48 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20230504012746/model_31.pt


Epoch 2/10:   0%|          | 0/31 [00:00<?, ?it/s]

2023-05-04 01:27:50 [info     ] CQL_20230504012746: epoch=2 step=62 epoch=2 metrics={'time_sample_batch': 0.000903790996920678, 'time_algorithm_update': 0.03586419936149351, 'temp_loss': 11.724647952664283, 'temp': 0.995309462470393, 'alpha_loss': -31.375866490025675, 'alpha': 1.0047140698279104, 'critic_loss': 3234.1929419732865, 'actor_loss': -25.97842062673261, 'time_step': 0.03689156809160786} step=62
2023-05-04 01:27:50 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20230504012746/model_62.pt


...

Epoch 9/10:   0%|          | 0/31 [00:00<?, ?it/s]

2023-05-04 01:27:59 [info     ] CQL_20230504012746: epoch=9 step=279 epoch=9 metrics={'time_sample_batch': 0.0009979432629000757, 'time_algorithm_update': 0.036409478033742594, 'temp_loss': 11.470276709525816, 'temp': 0.974076553698509, 'alpha_loss': -32.07833819235525, 'alpha': 1.0269050905781407, 'critic_loss': 2458.1851609753026, 'actor_loss': -35.75754042594664, 'time_step': 0.037541420229019656} step=279
2023-05-04 01:27:59 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20230504012746/model_279.pt


Epoch 10/10:   0%|          | 0/31 [00:00<?, ?it/s]

2023-05-04 01:28:00 [info     ] CQL_20230504012746: epoch=10 step=310 epoch=10 metrics={'time_sample_batch': 0.0007801055908203125, 'time_algorithm_update': 0.030945539474487305, 'temp_loss': 11.448118209838867, 'temp': 0.9710988998413086, 'alpha_loss': -32.18758958385837, 'alpha': 1.0301338165037093, 'critic_loss': 2336.2837937878026, 'actor_loss': -37.29167212209394, 'time_step': 0.031834071682345484} step=310
2023-05-04 01:28:00 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20230504012746/model_310.pt


[(1,
  {'time_sample_batch': 0.0009980278630410471,
   'time_algorithm_update': 0.08748865127563477,
   'temp_loss': 11.744194215343844,
   'temp': 0.9984005458893315,
   'alpha_loss': -31.277802190473004,
   'alpha': 1.0016018421419206,
   'critic_loss': 3721.7289094002017,
   'actor_loss': -10.316245063658684,
   'time_step': 0.0886292073034471}),
 (2,
  {'time_sample_batch': 0.000903790996920678,
   'time_algorithm_update': 0.03586419936149351,
   'temp_loss': 11.724647952664283,
   'temp': 0.995309462470393,
   'alpha_loss': -31.375866490025675,
   'alpha': 1.0047140698279104,
   'critic_loss': 3234.1929419732865,
   'actor_loss': -25.97842062673261,
   'time_step': 0.03689156809160786}),
 (3,
  {'time_sample_batch': 0.0009151427976546749,
   'time_algorithm_update': 0.03396520306987147,
   'temp_loss': 11.691720900997039,
   'temp': 0.9922334763311571,
   'alpha_loss': -31.483734069331998,
   'alpha': 1.0078407295288578,
   'critic_loss': 3119.4801301033267,
   'actor_loss': -26.3

In [15]:
import numpy as np
from d3rlpy.dataset import Episode, MDPDataset, Transition

OBSERVATION_SIZE = 768
ACTION_SIZE = 7
BATCH_SIZE = 64

all_observations = np.empty((0, OBSERVATION_SIZE))
all_actions = np.empty((0, ACTION_SIZE))
all_rewards = np.empty(0)
all_terminals = np.empty(0, dtype=bool)

for i in range(0,10):
    if i==7:
        continue
    observation = np.load(f'trajectories/all_observations_P00{i}.npy')
    action = np.load(f'trajectories/all_actions_P00{i}.npy')
    reward = np.load(f'trajectories/all_rewards_P00{i}.npy')
    terminals = np.load(f'trajectories/all_terminals_P00{i}.npy')

    all_observations = np.vstack((all_observations, observation))
    all_actions = np.vstack((all_actions, action))
    all_rewards = np.hstack((all_rewards, reward))
    all_terminals = np.hstack((all_terminals, terminals))

    print("All observations shape:", all_observations.shape)
    print("All actions shape:", all_actions.shape)
    print("All rewards shape:", all_rewards.shape)
    print("All terminals shape:", all_terminals.shape)
cql_dataset = MDPDataset(observations=all_observations,actions=all_actions,rewards=all_rewards,terminals=all_terminals,episode_terminals=all_terminals)

All observations shape: (1480, 768)
All actions shape: (1480, 7)
All rewards shape: (1480,)
All terminals shape: (1480,)
All observations shape: (2535, 768)
All actions shape: (2535, 7)
All rewards shape: (2535,)
All terminals shape: (2535,)
All observations shape: (3233, 768)
All actions shape: (3233, 7)
All rewards shape: (3233,)
All terminals shape: (3233,)
All observations shape: (4242, 768)
All actions shape: (4242, 7)
All rewards shape: (4242,)
All terminals shape: (4242,)
All observations shape: (5049, 768)
All actions shape: (5049, 7)
All rewards shape: (5049,)
All terminals shape: (5049,)
All observations shape: (5743, 768)
All actions shape: (5743, 7)
All rewards shape: (5743,)
All terminals shape: (5743,)
All observations shape: (6744, 768)
All actions shape: (6744, 7)
All rewards shape: (6744,)
All terminals shape: (6744,)
All observations shape: (7469, 768)
All actions shape: (7469, 7)
All rewards shape: (7469,)
All terminals shape: (7469,)
All observations shape: (8185, 7

In [4]:
from d3rlpy.algos import CQL
cql01 = CQL(use_gpu=False)
cql01.build_with_dataset(cql_dataset)
cql01.load_model('/home/ubuntu/camelmera/models/gym/multimodal/d3rlpy_logs/CQL_20230503011241/model_40.pt')

In [18]:
from d3rlpy.algos import CQL

cql = CQL.from_json('d3rlpy_logs/CQL_20230504012746/params.json')

# ready to load
cql.load_model('d3rlpy_logs/CQL_20230504012746/model_310.pt')

2023-05-04 01:34:42 [warning  ] Unused arguments are passed.   algorithm=CQL generated_maxlen=100000 real_ratio=1.0


In [19]:
# start training
cql.fit(cql_dataset,
        eval_episodes=None,
        n_epochs=30,
        scorers=None)

2023-05-04 01:35:10 [debug    ] RoundIterator is selected.
2023-05-04 01:35:10 [info     ] Directory is created at d3rlpy_logs/CQL_20230504013510
2023-05-04 01:35:10 [warning  ] Skip building models since they're already built.
2023-05-04 01:35:10 [info     ] Parameters are saved to d3rlpy_logs/CQL_20230504013510/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 0.0001, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': [0.9, 0.999], 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': [0.9, 0.999], 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_threshold': 10.0, 'batch_size': 256, 'conservative_weight': 5.0, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'critic_learn

Epoch 1/30:   0%|          | 0/31 [00:00<?, ?it/s]

2023-05-04 01:35:20 [info     ] CQL_20230504013510: epoch=1 step=31 epoch=1 metrics={'time_sample_batch': 0.0010161707478184854, 'time_algorithm_update': 0.3290915950652092, 'temp_loss': 11.405332565307617, 'temp': 0.9681331226902623, 'alpha_loss': -32.28081783171623, 'alpha': 1.0333774628177765, 'critic_loss': 2233.6551868069555, 'actor_loss': -38.560919607839274, 'time_step': 0.33023206649288056} step=31
2023-05-04 01:35:20 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20230504013510/model_31.pt


Epoch 2/30:   0%|          | 0/31 [00:00<?, ?it/s]

2023-05-04 01:35:27 [info     ] CQL_20230504013510: epoch=2 step=62 epoch=2 metrics={'time_sample_batch': 0.000981999981787897, 'time_algorithm_update': 0.2366661871633222, 'temp_loss': 11.369608602216166, 'temp': 0.9651807642752125, 'alpha_loss': -32.39259682932207, 'alpha': 1.0366356103650984, 'critic_loss': 2119.0245991368447, 'actor_loss': -40.40380416377898, 'time_step': 0.23776971909307665} step=62
2023-05-04 01:35:27 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20230504013510/model_62.pt


...

Epoch 29/30:   0%|          | 0/31 [00:00<?, ?it/s]

2023-05-04 01:38:50 [info     ] CQL_20230504013510: epoch=29 step=899 epoch=29 metrics={'time_sample_batch': 0.0009510055665046938, 'time_algorithm_update': 0.24219419879298057, 'temp_loss': 10.488471923335906, 'temp': 0.8900320722210792, 'alpha_loss': -35.31450283911921, 'alpha': 1.130293846130371, 'critic_loss': 518.6090343844506, 'actor_loss': -73.55164829377205, 'time_step': 0.24326179873558781} step=899
2023-05-04 01:38:50 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20230504013510/model_899.pt


Epoch 30/30:   0%|          | 0/31 [00:00<?, ?it/s]

2023-05-04 01:38:57 [info     ] CQL_20230504013510: epoch=30 step=930 epoch=30 metrics={'time_sample_batch': 0.0010590476374472341, 'time_algorithm_update': 0.23783430745524745, 'temp_loss': 10.458904973922238, 'temp': 0.8874029421037243, 'alpha_loss': -35.43197853334488, 'alpha': 1.1339762172391337, 'critic_loss': 490.8682034400202, 'actor_loss': -74.2077405375819, 'time_step': 0.23901430253059633} step=930
2023-05-04 01:38:57 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20230504013510/model_930.pt


[(1,
  {'time_sample_batch': 0.0010161707478184854,
   'time_algorithm_update': 0.3290915950652092,
   'temp_loss': 11.405332565307617,
   'temp': 0.9681331226902623,
   'alpha_loss': -32.28081783171623,
   'alpha': 1.0333774628177765,
   'critic_loss': 2233.6551868069555,
   'actor_loss': -38.560919607839274,
   'time_step': 0.33023206649288056}),
 (2,
  {'time_sample_batch': 0.000981999981787897,
   'time_algorithm_update': 0.2366661871633222,
   'temp_loss': 11.369608602216166,
   'temp': 0.9651807642752125,
   'alpha_loss': -32.39259682932207,
   'alpha': 1.0366356103650984,
   'critic_loss': 2119.0245991368447,
   'actor_loss': -40.40380416377898,
   'time_step': 0.23776971909307665}),
 (3,
  {'time_sample_batch': 0.0009399690935688634,
   'time_algorithm_update': 0.2364673614501953,
   'temp_loss': 11.33581853681995,
   'temp': 0.9622422341377505,
   'alpha_loss': -32.4925417746267,
   'alpha': 1.039909182056304,
   'critic_loss': 2022.7026839717741,
   'actor_loss': -41.97166701

In [21]:
# start training
cql.fit(cql_dataset,
        eval_episodes=None,
        n_epochs=10,
        scorers=None)

2023-05-04 01:43:38 [debug    ] RoundIterator is selected.
2023-05-04 01:43:38 [info     ] Directory is created at d3rlpy_logs/CQL_20230504014338
2023-05-04 01:43:38 [warning  ] Skip building models since they're already built.
2023-05-04 01:43:38 [info     ] Parameters are saved to d3rlpy_logs/CQL_20230504014338/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 0.0001, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': [0.9, 0.999], 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': [0.9, 0.999], 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_threshold': 10.0, 'batch_size': 256, 'conservative_weight': 5.0, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'critic_learn

Epoch 1/10:   0%|          | 0/31 [00:00<?, ?it/s]

2023-05-04 01:43:46 [info     ] CQL_20230504014338: epoch=1 step=31 epoch=1 metrics={'time_sample_batch': 0.0010034115083755986, 'time_algorithm_update': 0.26138419489706716, 'temp_loss': 9.549976348876953, 'temp': 0.8105977639075248, 'alpha_loss': -39.20918446202432, 'alpha': 1.255960222213499, 'critic_loss': 134.8759064212922, 'actor_loss': -90.1291038759293, 'time_step': 0.26251200706728045} step=31
2023-05-04 01:43:46 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20230504014338/model_31.pt


Epoch 2/10:   0%|          | 0/31 [00:00<?, ?it/s]

2023-05-04 01:43:53 [info     ] CQL_20230504014338: epoch=2 step=62 epoch=2 metrics={'time_sample_batch': 0.00093654663332047, 'time_algorithm_update': 0.24153282565455284, 'temp_loss': 9.522798476680633, 'temp': 0.8082602850852474, 'alpha_loss': -39.35825692453692, 'alpha': 1.2601512670516968, 'critic_loss': 130.87871649957472, 'actor_loss': -90.3958499047064, 'time_step': 0.24259078887201124} step=62
2023-05-04 01:43:53 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20230504014338/model_62.pt


...

Epoch 9/10:   0%|          | 0/31 [00:00<?, ?it/s]

2023-05-04 01:44:46 [info     ] CQL_20230504014338: epoch=9 step=279 epoch=9 metrics={'time_sample_batch': 0.0009593809804608745, 'time_algorithm_update': 0.23928150823039393, 'temp_loss': 9.333456808520902, 'temp': 0.7921131945425465, 'alpha_loss': -40.298756630189956, 'alpha': 1.2899631184916343, 'critic_loss': 108.35731334071005, 'actor_loss': -92.19059408864668, 'time_step': 0.2403572836229878} step=279
2023-05-04 01:44:46 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20230504014338/model_279.pt


Epoch 10/10:   0%|          | 0/31 [00:00<?, ?it/s]

2023-05-04 01:44:54 [info     ] CQL_20230504014338: epoch=10 step=310 epoch=10 metrics={'time_sample_batch': 0.0009909829785746912, 'time_algorithm_update': 0.24242529561442713, 'temp_loss': 9.30837578927317, 'temp': 0.7898380525650517, 'alpha_loss': -40.429276251023815, 'alpha': 1.2942882160986624, 'critic_loss': 106.09569254229146, 'actor_loss': -92.38607049757435, 'time_step': 0.2435398793989612} step=310
2023-05-04 01:44:54 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20230504014338/model_310.pt


[(1,
  {'time_sample_batch': 0.0010034115083755986,
   'time_algorithm_update': 0.26138419489706716,
   'temp_loss': 9.549976348876953,
   'temp': 0.8105977639075248,
   'alpha_loss': -39.20918446202432,
   'alpha': 1.255960222213499,
   'critic_loss': 134.8759064212922,
   'actor_loss': -90.1291038759293,
   'time_step': 0.26251200706728045}),
 (2,
  {'time_sample_batch': 0.00093654663332047,
   'time_algorithm_update': 0.24153282565455284,
   'temp_loss': 9.522798476680633,
   'temp': 0.8082602850852474,
   'alpha_loss': -39.35825692453692,
   'alpha': 1.2601512670516968,
   'critic_loss': 130.87871649957472,
   'actor_loss': -90.3958499047064,
   'time_step': 0.24259078887201124}),
 (3,
  {'time_sample_batch': 0.0009963973875968687,
   'time_algorithm_update': 0.23778229375039378,
   'temp_loss': 9.501792630841654,
   'temp': 0.8059299838158392,
   'alpha_loss': -39.50162998322518,
   'alpha': 1.2643601317559519,
   'critic_loss': 127.34921116982737,
   'actor_loss': -90.75865074896